In [ ]:
import yaml
import pandas as pd
import geopandas as gpd
import geohash2
from shapely import geometry
from pyproj import Transformer
from polygeohasher import polygeohasher # https://www.geospatialworld.net/blogs/polygeohasher-an-optimized-way-to-create-geohashes/



## Load Config

In [182]:
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [183]:
# Set Configs
filter_region: str = config['region']
paths: dict() = config['paths']
crs: dict() = config['crs']


In [ ]:
# Coord System Transformer
transformer_gps_to_osmeters = Transformer.from_crs(crs['latlon_gps'], crs['proj_os32'])

## Functions

In [163]:
def geohash_to_polygon(ghash: str) -> geometry.Polygon:

    lat_centroid, lng_centroid, lat_offset, lng_offset = geohash2.decode_exactly(ghash)

    corner_1 = transformer_gps_to_osmeters.transform(lat_centroid - lat_offset, lng_centroid - lng_offset)
    corner_2 = transformer_gps_to_osmeters.transform(lat_centroid + lat_offset, lng_centroid - lng_offset)
    corner_3 = transformer_gps_to_osmeters.transform(lat_centroid + lat_offset, lng_centroid + lng_offset)
    corner_4 = transformer_gps_to_osmeters.transform(lat_centroid - lat_offset, lng_centroid + lng_offset)

    polygon = geometry.Polygon([corner_1, corner_2, corner_3, corner_4, corner_1])

    return polygon

## Main

In [ ]:
# Load Polygon to GeoDataFrame
gdf_Polygon: gpd.GeoDataFrame = gpd.read_file(paths['area_polygon'], crs=crs['meters'])

In [ ]:
# Filter by Region
gdf_Polygon = gdf_Polygon[gdf_Polygon['Region'] == filter_region]

In [ ]:
# Convert CRS to GPS
gdf_Polygon_GPS = gdf_Polygon.to_crs(crs['latlon_gps'])

In [138]:
# Melt to single polygon
union_geom = gdf_Polygon_GPS['geometry'].unary_union

In [139]:
df_Union = pd.DataFrame({'id':'0', 'geometry': union_geom}, index=[0])

In [144]:
gdf_Union = gpd.GeoDataFrame(df_Union, crs=crs['latlon_gps'])

In [145]:
gdf_poly_hashed = polygeohasher.create_geohash_list(gdf=gdf_Union, geohash_level=8, inner=False)

In [158]:
hash_set = set(gdf_poly_hashed['geohash_list'].to_list()[0])

In [170]:
hash_dict = [{'GeoHash': n, 'geometry': geohash_to_polygon(n)} for n in hash_set]

In [172]:
gdf_hash_dict = gpd.GeoDataFrame(data=hash_dict, crs=crs['meters'])

In [178]:
gdf_hash_dict.set_index('GeoHash', inplace=True)

In [185]:
gdf_hash_dict.to_feather(path=paths['save_as']+'.feather', index=True)